## D212 Data Mining 2 PA 2
##### Submitted By Edwin Perry
### Table of Contents
<ol>
    <li><a href="#A">Research Question</a></li>
    <li><a href="#B">Technique Justification</a></li>
    <li><a href="#C">Data Preparation</a></li>
    <li><a href="#D">Analysis</a></li>
    <li><a href="#E">Data Summary and Implications</a></li>
    <li><a href="#F">Panopto Video</a></li>
</ol>


<h4 id="A">Research Question</h4>
<h5>Question</h5>
<p>The research question for this analysis is "Can customer churn be predicted by a decision tree after Principal Component Analysis (PCA) is used to reduce dimensionality?" This question helps to drive the success of the business, as it enables the business to understand customer sentiment and likelihood of losing a customer, potentially helping the business to reduce customers lost and prepare earlier for new customer acquisition</p>
<h5>Goal of Analysis</h5>
<p>The goal of the analysis is to create a model that can accurately predict customer churn. The telecommunications industry has a high customer acquisition cost, which means identifying customers that are likely to churn in the future allow the business to take actions to maximize revenue, ideally by finding a way to keep customers, as the cost to acquire a new customer is usually 10 times the cost to retain a customer.</p>
<h4 id="B">Technique Justification</h4>
<h5>Method Explanation</h5>
Principal Component Analysis is a useful technique in data analysis. Specifically, PCA allows an analyst to reduce the number of dimensions being considered by a model, allowing the analyst to extract composite features. These composite features allow for different dimensions to be considered as part of one larger dimension that can be more useful at deriving insights within the analysis. My anticipated outcome of PCA would be a significant reduction to the number of dimensions being considered, using the explained_variance_ratio attribute, which would allow me to have a more streamlined and effective decision tree than the analysis would otherwise be able to obtain</p>
<h5>Assumptions of PCA</h5>
<p>One of the assumptions that I must make for PCA to be valid is that there are no outliers (or that the impact of outliers in minimal). PCA uses eigenvectors to determine principal components, and as such, a single or handful of extreme values can lead to misleading results. As such, we will be filtering outliers in our analysis, to ensure that the PCA can be validly performed.</p>
<h4 id="C">Data Preparation</h4>
<h5>Continuous Variable Identification</h5>
<p>The PCA dimension reduction will be performed speccifically on continuous variables. The ones relevant for the analysis are as follows:
<ul>
<li>Tenure: The number of months that the customer has been a customer of the telecommunications company</li>
<li>Income: The annual income in dollars of the customer</li>
<li>Bandwidth_gb_year: The amount of data a customer uses in a year</li>
<li>Outage_sec_perweek: The average number of seconds the customer's neighborhood experiences on a weekly basis</li>
</ul>
</p>
<h5>Standardization</h5>